### Importar Librerías 

In [1]:
import pandas as pd
import numpy as np
import os
from functools import partial, reduce
import operator
import re
import time
from bs4 import BeautifulSoup
from collections import Counter

In [2]:
def getAllPaths(data_directory):
    #Input: directorio de los archivos.
    #Return: Arreglo con las rutas de los archivos.
    file_name = [os.path.join(data_directory,f) 
                 for f in os.listdir(data_directory)
                 if f.endswith(".csv")]
    return file_name

In [3]:
def clean_tweet(tweet):
    try:
        if(isinstance(tweet[0],str)):
            tweet = BeautifulSoup(tweet[0], "lxml").get_text()
            # Removing the @
            tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
            # Removing the URL links
            tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
            # Keeping only letters
            tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
            # Removing additional whitespaces
            tweet = re.sub(r" +", ' ', tweet)
    except:
        tweet=""
    return tweet

### Cargo todos los dataset de pruebas.

In [4]:
PATHS_CSV = getAllPaths("../data_sophia2/")
PATH_DF = pd.DataFrame(PATHS_CSV)
PATH_DF.columns=["PATH_FILES"]
PATH_DF.head()

,PATH_FILES
0,../data_sophia2/chile_2020-12.csv
1,../data_sophia2/chile_2021-03.csv
2,../data_sophia2/chile_2021-04.csv
3,../data_sophia2/chile_2020-11.csv
4,../data_sophia2/chile_2021-01.csv


### Nos quedamos solo con los textos

In [5]:
def cargarDF(x,df,col="PATH_FILES"):
    data =(df.iloc[x,0:1].values[0])
    return pd.read_csv(data)

def reducirDF(x,df,colname):
    return df[x].iloc[:,5:6]

def dropNA(x,df):
    data = df.copy()
    return data[x].dropna()

### cargar los datasets

In [6]:
df__sophia2 = list(map(partial(cargarDF, df=PATH_DF), range(PATH_DF.shape[0])))

In [7]:
df_sample_sophia2 =list(map(partial(reducirDF, df=df__sophia2,colname="text"),range(len(df__sophia2))))
df_sample_sophia2[2]

,text
0,En respuesta a los anuncios realizados hoy por...
1,La comisión de Hacienda del Senado aprobó en...
2,Una de las indicaciones que ha encendido las a...
3,La comisión de Hacienda del Senado aprobó es...
4,La comisión de Constitución de la Cámara hoy s...
...,...
1988,Este lunes Nicolás Jarry (507°) debía enfrent...
1989,Este lunes se volvió a actualizar el ranking ...
1990,golagol('/especiales/2020/deportes/gol-gol/Cha...
1991,golagol('/especiales/2021/deportes/gol-gol/Cop...


In [8]:
df_sophia2_subset_pd = list(map(partial(dropNA,df=df_sample_sophia2),
                                range(len(df_sample_sophia2))))



In [9]:
tam = len(df_sophia2_subset_pd)
df_sophia2_subset = [pd.DataFrame(df_sophia2_subset_pd[i]) for i in range(tam)]

In [10]:
display([df_sophia2_subset[i].isna().sum() for i in range(len(df_sophia2_subset))])

[text    0
 dtype: int64,
 text    0
 dtype: int64,
 text    0
 dtype: int64,
 text    0
 dtype: int64,
 text    0
 dtype: int64,
 text    0
 dtype: int64,
 text    0
 dtype: int64,
 text    0
 dtype: int64]

In [11]:
print(f"Cantidad total de textos a procesar:  {np.sum([ df__sophia2[i].shape[0] for i in range(len(PATH_DF))])}")

Cantidad total de textos a procesar:  69702


In [12]:
#del df__sophia2, PATH_CF,df_sample_sophia2, stop_row

In [13]:
#x=[df_sophia2_subset[k].to_csv(f"../data_subset/df_sample_sophia2{k}.csv") for k in range(len(df_sophia2_subset))]

In [14]:
ITERATOR_DF = list(range(len(df_sophia2_subset))) # tamaño del array de datasets.
print(f"Cantidad de DataFrames: {len(ITERATOR_DF)}")

Cantidad de DataFrames: 8


### Generamos los Matches con Spacy.

In [16]:
#!pip install -U spacy-nightly
#!python -m spacy download es_core_news_sm
#!pip install -U pip setuptools wheel
#!pip uninstall en-core-web-sm

In [15]:
import spacy 
from spacy.matcher import Matcher
gpu= spacy.prefer_gpu()
#nlp = spacy.load('es_core_news_sm')
#nlp = spacy.load('en_core_web_lg')
nlp = spacy.load('es_dep_news_trf')
matcher = Matcher(nlp.vocab)
pattern_1 = [{"POS": "NOUN"},{"LOWER": "de"}, {"POS": "NOUN"}]
matcher.add("NOUN-de-NOUN", [pattern_1])
pattern_2 = [{"POS": "NOUN"}, {"POS": "ADJ"}]
matcher.add("NOUN-ADJ", [pattern_2])

In [16]:
def counterReduce(matches, doc):
        start = matches[1]
        end = matches[2]
        span = doc[start:end]
        return span.text

def getText(x,df,nlp,matcher):
    #x : iterador de los textos del dataset en posicion index
    #index: indice del dataset dentro del array de dataset df_sophia2_subset
    # df : Dataframe df_sophia2_subset
    #return : span for all matcher 
    #print(df.loc[x,["text"]].values[0])
    text_atomic = df.loc[x,["text"]].values[0]
    doc_x = nlp(text_atomic)
    matches = matcher(doc_x)
    span_reduce = list(map(partial(counterReduce, doc=doc_x),matches))
    #Se podría clasificar ahora mismo. 
    return Counter(span_reduce)


def launchTask(x,df,nlp,matcher):
    #index_df: iterador del array de dataset
    #df: Dataframe df_sophia2_subset
    #return: lista de arrays que contiene las construcciones terminologícas

    data_clean = list(map(clean_tweet , df[x].values))
    df_clean = pd.DataFrame(data_clean)
    df_clean.columns = ["text"]
    tam_df_current=len(df_clean)
    #print(len(df[x]),tam_df_current)
    term = list(map(partial(getText, df=df_clean,nlp=nlp,matcher=matcher),range(tam_df_current) ))
    Concepts = reduce(operator.add, term )
    print("Terminado")
    result = pd.DataFrame(Concepts.most_common(), columns =['Concept', 'Frequency'])
    result.to_csv(f"./terminology_DF{x}_31_10_2021.csv")
    return Concepts

### Obtenemos una collections de los conceptos.

In [19]:
df_sophia2_subset[7].loc[350,"text"]

"Un descenso generalizado de casos activos registró el país durante los últimos días, según reveló este miércoles el  97° informe epidemiológico  publicado por el Minsal, que da cuenta del avance de la pandemia del coronavirus en Chile. De acuerdo al reporte,  a nivel nacional hay un total de 25.410 personas contagiantes, lo que representa una baja de 1.095 casos  en comparación con el informe anterior, publicado el sábado, cuando había 26.505 activos. RelacionadasDetalle('1013120','1013187'); Asimismo, el estudio indica que  15 de las 16 regiones del país registraron bajas en este indicador,  siendo  Los Lagos  la que anotó la mayor disminución, con 391 casos activos menos. Segunda se ubica la  Región Metropolitana,  con una baja de 163 contagios, seguida por  Tarpacá  (-117),  Antofagasta  (-110),  O'Higgins  (-98),  Ñuble  (-71),  Valparaíso  (-37),  Biobío  (-34) y  Arica  (-29). Mientras que las regiones que menos bajaron sus casos activos fueron el  Maule  (-25),  Atacama  (-21),

In [20]:
%%time
#from multiprocessing import Pool
#p=Pool(6)
Concept_freq = list(map(partial(launchTask,df=df_sophia2_subset,nlp=nlp,matcher=matcher),ITERATOR_DF))

Token indices sequence length is longer than the specified maximum sequence length for this model (617 > 512). Running this sequence through the model will result in indexing errors


Terminado
Terminado
Terminado
Terminado
Terminado
Terminado
Terminado
Terminado
CPU times: user 2d 4h 33min 56s, sys: 30min 57s, total: 2d 5h 4min 54s
Wall time: 9h 4min


### Reducimos el contador de conceptos.

In [21]:
concept_freq_total = reduce(operator.add,Concept_freq)

In [ ]:
Concept_freq

In [ ]:
concept_freq_total

In [22]:
data = concept_freq_total.most_common()
result = pd.DataFrame(data, columns =['Concept', 'Frequency'])
result.to_csv("./terminology_summary_concepts.csv")